In [15]:
import pandas as pd
import numpy as np
from scipy import spatial
from gensim.models import word2vec
import pickle

vector_size = 250

In [16]:
'''
csv読み込み
store_idをindexに変更
'''

df = pd.read_csv('../csv/review_wakati.csv')
df = df.drop(['store_id'],axis=1)
df = df.rename(columns={'Unnamed: 0':'store_id'}).set_index('store_id')

In [17]:
'''
pickleで学習済みreview達のvector読み出し
'''

f = open("../pickle/skip_list.txt","rb")
skip_list = pickle.load(f)
f.close()

In [18]:
'''
学習済みモデルの読み出し
'''

model = word2vec.Word2Vec.load('../models/skip_w2v.model')

In [19]:
'''
ここまで準備
'''

'\nここまで準備\n'

In [20]:
def sentence_similarity(vec1, vec2):
    '''
    二つの文章のコサイン類似度を返す
    '''
    return 1 - spatial.distance.cosine(vec1, vec2)

In [8]:
def avg_feature_vectors(words, model):
    feature_vec = np.zeros(vector_size, dtype='float32')
    for word in words:
        try:
            feature_vec = np.add(feature_vec, model[word])
        except KeyError:
            pass

    return feature_vec

In [24]:
'''
store_idをkeyに指定文書と各行のreviewのコサイン類似度をスコアとして入れる
'''

result = {}
keyword = avg_feature_vectors(['濃厚','太麺','つけ麺'], model)
for i, row in df.iterrows():
    if row.station not in ['新宿', '西新宿','新宿三丁目']:
        continue
    score = sentence_similarity(keyword, skip_list[i])
    print(score*10*1.2, row.score)
    result[i] = score*10*1.2 + row.score


7.734234097821377 3.84
7.68510484794386 3.76
7.729056967152828 3.7
8.014446983882227 3.7
7.816509343465975 3.69
8.034184900172395 3.67
8.515252203296233 3.63
7.309879055497888 3.58
7.839624169916551 3.56
7.202644243942077 3.54
7.437389930795904 3.53
7.8291330732454245 3.53
/Users/kazuki_takahashi/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


In [25]:
'''
scoreの高い順にsort
'''

score_sorted = sorted(result.items(), key=lambda x:x[1], reverse=True)

In [27]:
'''
score高い10件を書き出し
'''

for t in score_sorted[:10]:
    row = df[df.index == t[0]]
    name, score, station = row.store_name.values[0], row.score.values[0], row.station.values[0]
    print('店名:{}, 食べログスコア:{}, 独自スコア:{:.2f}, 最寄駅:{}'.format(name,score,t[1],station))
    

店名:百日紅, 食べログスコア:3.63, 独自スコア:12.15, 最寄駅:新宿三丁目
店名:煮干中華そば鈴蘭, 食べログスコア:3.7, 独自スコア:11.71, 最寄駅:新宿三丁目
店名:麺屋 翔 みなと, 食べログスコア:3.67, 独自スコア:11.70, 最寄駅:西新宿
店名:らぁ麺 はやし田 新宿本店, 食べログスコア:3.84, 独自スコア:11.57, 最寄駅:新宿三丁目
店名:メンショー サン フランシスコ, 食べログスコア:3.69, 独自スコア:11.51, 最寄駅:新宿
店名:麺屋海神, 食べログスコア:3.76, 独自スコア:11.45, 最寄駅:新宿
店名:すごい煮干ラーメン凪 新宿ゴールデン街店本館, 食べログスコア:3.7, 独自スコア:11.43, 最寄駅:新宿三丁目
店名:らーめん 北の大地, 食べログスコア:3.56, 独自スコア:11.40, 最寄駅:新宿三丁目
店名:らぁ麺 くろ渦, 食べログスコア:3.53, 独自スコア:11.36, 最寄駅:新宿三丁目
店名:町田家 新宿南口店, 食べログスコア:3.53, 独自スコア:10.97, 最寄駅:新宿
